# instalaci√≥n de las librerias 

In [1]:
!pip install pandas numpy




[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

import numpy as np

import sqlite3

# Carga de base de datos 


## Configuraci√≥n Inicial y Carga de Datos (Optimizado)

Este bloque carga los archivos CSV principales del proyecto y aplica filtros para centrarse en las temporadas entre **2013 y 2024**. Tambi√©n incluye la normalizaci√≥n de nombres para facilitar futuras uniones.

---

### **Archivos Utilizados:**

- **`draft_history.csv`**: Registros de jugadores seleccionados en el draft.
- **`team_info_common.csv`**: Informaci√≥n b√°sica sobre equipos.
- **`game.csv`**: Detalles de cada partido, incluyendo fecha y equipos.
- **`game_summary.csv`**: Estad√≠sticas generales por partido.
- **`other_stats.csv`**: M√©tricas avanzadas como rebotes y puntos en la pintura.
- **`play_by_play.csv`**: Jugadas individuales por partido.
- **`player.csv`**: Identificadores √∫nicos de jugadores y sus nombres completos.
- **`common_player_info.csv`**: Datos personales de cada jugador, como edad y nacionalidad.
- **`mvps.csv`**: Datos de premios al jugador m√°s valioso (MVP) por temporada.
- **`salarios_nba_final_player.csv`**: Informaci√≥n salarial de jugadores para m√∫ltiples temporadas.

---

### **Procesos Aplicados:**

1. **Normalizaci√≥n de Nombres:** Todos los nombres se convirtieron a min√∫sculas para asegurar consistencia en las uniones.
2. **Fusi√≥n de Tablas de Jugadores:** Se cre√≥ un DataFrame maestro (**`player_master_df`**) combinando **`player.csv`** y **`common_player_info.csv`**.
3. **Filtrado por Temporada:** Se aplic√≥ un filtrado temprano para limitar los datos a las temporadas **2013-2024**.
4. **Filtrado de Jugadores Relevantes:** Se identificaron los jugadores que aparecen en el draft o en jugadas registradas.
5. **Integraci√≥n del Archivo de Salarios:** Se incorpor√≥ el nuevo archivo **`salarios_nba_final_player.csv`** con optimizaci√≥n de tipos de datos.

---

Este bloque establece las bases para los an√°lisis posteriores, asegurando que los datos est√©n correctamente estructurados y filtrados antes de continuar con las limpiezas y uniones m√°s avanzadas.







In [3]:
# Configuraci√≥n Inicial y Carga de Datos
import pandas as pd
import numpy as np

# Par√°metros de filtrado
A√ëO_MIN, A√ëO_MAX = 2013, 2024
ruta_base = "C:/Henry/Proyecto_final/NBA_Datos/csv_Tf_henry/"

# Archivos para cargar  
files = {
    'draft': 'draft_history.csv',
    'game': 'game.csv',
    'game_summary': 'game_summary.csv',
    'other_stats': 'other_stats.csv',
    'play_by_play': 'play_by_play.csv',
    'player': 'player.csv',
    'common_player_info': 'common_player_info.csv',
    'mvp': 'mvps.csv',
    'salarios': 'salarios_nba_final_player.csv'
}

# Cargar todos los archivos en un solo paso
dataframes = {name: pd.read_csv(ruta_base + file) for name, file in files.items()}

# Filtrado por Temporada (2013-2024)
for key in ['draft', 'game', 'mvp', 'salarios']:
    if 'season' in dataframes[key].columns:
        dataframes[key] = dataframes[key][dataframes[key]['season'].between(A√ëO_MIN, A√ëO_MAX)]
    elif 'Year' in dataframes[key].columns:
        dataframes[key] = dataframes[key][dataframes[key]['Year'].between(A√ëO_MIN, A√ëO_MAX)]

# Resumen de datos cargados
for name, df in dataframes.items():
    print(f"- {name.capitalize()}: {df.shape}")







- Draft: (596, 14)
- Game: (65698, 55)
- Game_summary: (58110, 14)
- Other_stats: (28271, 26)
- Play_by_play: (13592899, 34)
- Player: (4831, 5)
- Common_player_info: (4171, 33)
- Mvp: (143, 22)
- Salarios: (1350, 12)


In [4]:
#  Normalizaci√≥n de Nombres y Tipos para Fusi√≥n
dataframes['player']['full_name'] = dataframes['player']['full_name'].str.lower()
dataframes['player']['person_id'] = dataframes['player']['person_id'].astype(str)
dataframes['common_player_info']['person_id'] = dataframes['common_player_info']['person_id'].astype(str)

#  Fusi√≥n de DataFrames
player_master_df = pd.merge(
    dataframes['player'], 
    dataframes['common_player_info'], 
    on='person_id', 
    how='left', 
    suffixes=('_player', '_common')
)

#  Eliminaci√≥n de Columnas Redundantes
columnas_redundantes = [
    'full_name_player', 'display_first_last', 
    'first_name_common', 'last_name_common', 
    'display_last_comma_first', 'display_fi_last'
]

# Verificar que las columnas redundantes existan antes de eliminarlas
columnas_redundantes_existentes = [col for col in columnas_redundantes if col in player_master_df.columns]
player_master_df.drop(columns=columnas_redundantes_existentes, inplace=True)

#  Verificaci√≥n Inicial
print("\n Resumen de 'player_master' despu√©s de la fusi√≥n:")
print(player_master_df.info())
print(player_master_df.head(10))

#  A√±adir el DataFrame fusionado al diccionario principal
dataframes['player_master'] = player_master_df







 Resumen de 'player_master' despu√©s de la fusi√≥n:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4831 entries, 0 to 4830
Data columns (total 32 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   person_id                         4831 non-null   object 
 1   full_name                         4831 non-null   object 
 2   first_name_player                 4825 non-null   object 
 3   last_name_player                  4831 non-null   object 
 4   is_active                         4831 non-null   int64  
 5   player_slug                       4171 non-null   object 
 6   birthdate                         4171 non-null   object 
 7   school                            4156 non-null   object 
 8   country                           4170 non-null   object 
 9   last_affiliation                  4171 non-null   object 
 10  height                            4075 non-null   object 
 11  weight          

In [5]:
# Crear variables separadas para cada DataFrame filtrado
draft_filtrado = dataframes['draft']
game_filtrado = dataframes['game']
game_summary_filtrado = dataframes['game_summary']
other_stats_filtrado = dataframes['other_stats']
play_by_play_filtrado = dataframes['play_by_play']
player_master_filtrado= dataframes['player_master']
MVP_filtrado = dataframes['mvp']
salarios_filtrado = dataframes['salarios']
common_info_filtrado = dataframes['common_player_info']

# Verificaci√≥n r√°pida
print("üìã Resumen de Datos Cargados:")
for name, df in dataframes.items():
    print(f"- {name.capitalize()}: {df.shape}")

üìã Resumen de Datos Cargados:
- Draft: (596, 14)
- Game: (65698, 55)
- Game_summary: (58110, 14)
- Other_stats: (28271, 26)
- Play_by_play: (13592899, 34)
- Player: (4831, 5)
- Common_player_info: (4171, 33)
- Mvp: (143, 22)
- Salarios: (1350, 12)
- Player_master: (4831, 32)


In [6]:
# Verificar que el archivo se carg√≥ correctamente
print("\nüìã Archivos cargados:")
for name, df in dataframes.items():
    print(f"- {name}: {df.shape}")


üìã Archivos cargados:
- draft: (596, 14)
- game: (65698, 55)
- game_summary: (58110, 14)
- other_stats: (28271, 26)
- play_by_play: (13592899, 34)
- player: (4831, 5)
- common_player_info: (4171, 33)
- mvp: (143, 22)
- salarios: (1350, 12)
- player_master: (4831, 32)


In [7]:
#  Verificaci√≥n de Duplicados y Nulos

# Verificar duplicados por 'person_id'
duplicados = player_master_df.duplicated(subset=['person_id'], keep=False)
print(f"\n Duplicados encontrados: {duplicados.sum()}")
if duplicados.sum() > 0:
    print(player_master_df[duplicados].head(10))

# Resumen de nulos
nulos = player_master_df.isnull().sum()
print("\n Resumen de Valores Nulos:")
print(nulos[nulos > 0])



 Duplicados encontrados: 0

 Resumen de Valores Nulos:
first_name_player                      6
player_slug                          660
birthdate                            660
school                               675
country                              661
last_affiliation                     660
height                               756
weight                               760
season_exp                           660
jersey                              1640
position                             723
rosterstatus                         660
games_played_current_season_flag     660
team_id                              660
team_name                           1362
team_abbreviation                   1362
team_code                           1362
team_city                           1362
playercode                           661
from_year                            675
to_year                              675
dleague_flag                         660
nba_flag                             660
g

In [8]:
#   Manejo de Nulos y Optimizaci√≥n de Datos

import re

#   Conversi√≥n de alturas a cent√≠metros (Ajustado)
def convertir_altura_cm(altura):
    if pd.isna(altura) or not isinstance(altura, str):
        return np.nan
    try:
        pies, pulgadas = map(int, altura.split('-'))
        return pies * 30.48 + pulgadas * 2.54
    except ValueError:
        return np.nan

# Aplicar la conversi√≥n de altura solo a datos v√°lidos
player_master_df['height'] = player_master_df['height'].apply(convertir_altura_cm)

#   Manejo de valores nulos cr√≠ticos (sin SettingWithCopyWarning)
player_master_df.loc[:, 'weight'] = player_master_df['weight'].fillna(player_master_df['weight'].mean()).round().astype('Int64')
player_master_df.loc[:, 'position'] = player_master_df['position'].fillna('Unknown')
player_master_df.loc[:, 'team_name'] = player_master_df['team_name'].fillna('No Team')
player_master_df.loc[:, 'team_abbreviation'] = player_master_df['team_abbreviation'].fillna('NT')
player_master_df.loc[:, 'team_code'] = player_master_df['team_code'].fillna('NT')
player_master_df.loc[:, 'team_city'] = player_master_df['team_city'].fillna('No City')

#   Optimizaci√≥n de tipos de datos
player_master_df['is_active'] = player_master_df['is_active'].astype('Int64')
player_master_df['season_exp'] = player_master_df['season_exp'].astype('Int64')
player_master_df['weight'] = player_master_df['weight'].astype('Int64')

#   Resumen final despu√©s de la limpieza
print("\n Resumen despu√©s de limpiar nulos y optimizar datos:")
print(player_master_df.info())
print(player_master_df.head(10))




 Resumen despu√©s de limpiar nulos y optimizar datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4831 entries, 0 to 4830
Data columns (total 32 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   person_id                         4831 non-null   object 
 1   full_name                         4831 non-null   object 
 2   first_name_player                 4825 non-null   object 
 3   last_name_player                  4831 non-null   object 
 4   is_active                         4831 non-null   Int64  
 5   player_slug                       4171 non-null   object 
 6   birthdate                         4171 non-null   object 
 7   school                            4156 non-null   object 
 8   country                           4170 non-null   object 
 9   last_affiliation                  4171 non-null   object 
 10  height                            4075 non-null   float64
 11  weight        

In [9]:
#  Verificaci√≥n de Consistencia y An√°lisis de Valores At√≠picos

#   Identificar valores √∫nicos y posibles inconsistencias
columnas_revisar = ['position', 'team_name', 'team_abbreviation', 'team_code', 'team_city']

for columna in columnas_revisar:
    valores_unicos = player_master_df[columna].dropna().unique()
    print(f"\n Valores √∫nicos en '{columna}': {len(valores_unicos)}")
    print(valores_unicos)

# Resumen r√°pido de valores faltantes despu√©s de la optimizaci√≥n
print("\n Resumen de Valores Nulos (Post-Optimizaci√≥n):")
print(player_master_df.isnull().sum())


 Valores √∫nicos en 'position': 8
['Forward' 'Unknown' 'Center' 'Forward-Guard' 'Guard' 'Center-Forward'
 'Guard-Forward' 'Forward-Center']

 Valores √∫nicos en 'team_name': 51
['Trail Blazers' 'No Team' 'Lakers' 'Kings' 'Grizzlies' 'Warriors'
 'Nationals' 'Ironmen' 'Raptors' 'Clippers' 'Knicks' 'Magic' 'Rockets'
 'Suns' 'Pistons' 'Nuggets' 'Nets' 'Heat' 'Cavaliers' 'Jazz' 'Mavericks'
 'Pelicans' 'Bullets' '76ers' 'Spurs' 'Bucks' 'Timberwolves' 'Bulls'
 'Pacers' 'Hawks' 'Celtics' 'Wizards' 'Hornets' 'Royals' 'Falcons'
 'Braves' 'SuperSonics' 'Stags' 'Bombers' 'Olympians' 'Capitols' 'Rebels'
 'Blackhawks' 'Packers' 'Bobcats' 'Jets' 'Thunder' 'Steamrollers'
 'Redskins' 'Zephyrs' 'Huskies']

 Valores √∫nicos en 'team_abbreviation': 70
['POR' 'NT' 'LAL' 'SAC' 'VAN' 'GOS' 'PHI' 'PIT' 'TOR' 'LAC' 'NYK' 'ORL'
 'SDR' 'PHX' 'DET' 'MEM' 'DEN' 'NJN' 'MIA' 'CLE' 'UTA' 'DAL' 'NOP' 'WAS'
 'BKN' 'SAS' 'MIL' 'MIN' 'CHI' 'SFW' 'IND' 'HOU' 'ATL' 'STL' 'KCK' 'BOS'
 'PHW' 'NOH' 'FTW' 'CIN' 'BAL' 'DEF' 'B

## **Estandarizaci√≥n de Posiciones, Nombres de Equipos y Abreviaturas**

Para asegurar consistencia en el an√°lisis de jugadores y equipos, se realiz√≥ una estandarizaci√≥n de nombres, posiciones y abreviaturas, eliminando variantes hist√≥ricas y asegurando uniformidad en los datos.

### ** Mapeo de Posiciones**

Se agruparon posiciones similares para simplificar el an√°lisis:

```python
# Mapeo para estandarizar posiciones
posicion_mapeo = {
    'Forward-Guard': 'Forward',
    'Guard-Forward': 'Guard',
    'Center-Forward': 'Center',
    'Forward-Center': 'Forward',
    'Guard-Forward': 'Guard',
    'Forward-Guard': 'Forward'
}

player_master_df['position'] = player_master_df['position'].replace(posicion_mapeo)
```

### ** Mapeo de Nombres de Equipos**

Se ajustaron los nombres de equipos hist√≥ricos para reflejar sus equivalentes modernos:

```python
# Mapeo para estandarizar nombres de equipos
equipo_mapeo = {
    'SuperSonics': 'Thunder',  # Seattle SuperSonics ahora Oklahoma City Thunder
    'Bullets': 'Wizards',  # Washington Bullets ahora Washington Wizards
    'Braves': 'Clippers',  # Buffalo Braves ahora Los Angeles Clippers
    'Nationals': '76ers',  # Syracuse Nationals ahora Philadelphia 76ers
    'Royals': 'Kings',  # Cincinnati Royals ahora Sacramento Kings
    'Packers': 'Hawks',  # Tri-Cities Blackhawks ahora Atlanta Hawks
    'Huskies': 'Raptors',  # Toronto Huskies ahora Toronto Raptors
    'Ironmen': 'Pistons',  # Fort Wayne Ironmen ahora Detroit Pistons
    'No Team': 'Free Agent'
}

player_master_df['team_name'] = player_master_df['team_name'].replace(equipo_mapeo)
```

### ** Ventajas del Enfoque:**

- Consistencia en los datos, simplificando an√°lisis posteriores.
- Facilita las uniones con otras fuentes de datos que usan convenciones modernas.
- Reduce el riesgo de errores al comparar estad√≠sticas hist√≥ricas.
- Mantenimiento simplificado mediante mapeos centralizados.




In [10]:
# Mapeo para estandarizar posiciones
posicion_mapeo = {
    'Forward-Guard': 'Forward',
    'Guard-Forward': 'Guard',
    'Center-Forward': 'Center',
    'Forward-Center': 'Forward',
    'Guard-Forward': 'Guard',
    'Forward-Guard': 'Forward'
}

player_master_df['position'] = player_master_df['position'].replace(posicion_mapeo)

# Mapeo para estandarizar nombres de equipos
equipo_mapeo = {
    'SuperSonics': 'Thunder',  # Seattle SuperSonics ahora Oklahoma City Thunder
    'Bullets': 'Wizards',  # Washington Bullets ahora Washington Wizards
    'Braves': 'Clippers',  # Buffalo Braves ahora Los Angeles Clippers
    'Nationals': '76ers',  # Syracuse Nationals ahora Philadelphia 76ers
    'Royals': 'Kings',  # Cincinnati Royals ahora Sacramento Kings
    'Packers': 'Hawks',  # Tri-Cities Blackhawks ahora Atlanta Hawks
    'Huskies': 'Raptors',  # Toronto Huskies ahora Toronto Raptors
    'Ironmen': 'Pistons',  # Fort Wayne Ironmen ahora Detroit Pistons
    'No Team': 'Free Agent'
}

player_master_df['team_name'] = player_master_df['team_name'].replace(equipo_mapeo)

# Mapeo para abreviaturas (simplificado)
abreviatura_mapeo = {
    'NT': 'FA',  # Free Agent
    'POR': 'POR', 'LAL': 'LAL', 'SAC': 'SAC', 'VAN': 'VAN', 'GOS': 'GSW',
    'PHX': 'PHX', 'DET': 'DET', 'DEN': 'DEN', 'NYN': 'NYK', 'MIA': 'MIA',
    'CLE': 'CLE', 'UTA': 'UTA', 'DAL': 'DAL', 'NOP': 'NOP', 'WAS': 'WAS',
    'BKN': 'BKN', 'SAS': 'SAS', 'MIN': 'MIN', 'CHI': 'CHI', 'ATL': 'ATL',
    'BOS': 'BOS', 'OKC': 'OKC', 'TOR': 'TOR', 'CHA': 'CHA', 'HOU': 'HOU',
    'MEM': 'MEM', 'IND': 'IND', 'MIL': 'MIL'
}

player_master_df['team_abbreviation'] = player_master_df['team_abbreviation'].replace(abreviatura_mapeo)

# Ajustar los c√≥digos de equipo
codigo_equipo_mapeo = {
    'NT': 'FA', 'blazers': 'POR', 'lakers': 'LAL', 'kings': 'SAC', 
    'grizzlies': 'MEM', 'warriors': 'GSW', 'sixers': 'PHI', 'ironmen': 'DET'
}

player_master_df['team_code'] = player_master_df['team_code'].replace(codigo_equipo_mapeo)

# Verificaci√≥n r√°pida
print("\n Valores √∫nicos despu√©s de la estandarizaci√≥n:")
print("Posiciones:", player_master_df['position'].unique())
print("Nombres de equipos:", player_master_df['team_name'].unique())
print("Abreviaturas de equipos:", player_master_df['team_abbreviation'].unique())
print("C√≥digos de equipos:", player_master_df['team_code'].unique())


 Valores √∫nicos despu√©s de la estandarizaci√≥n:
Posiciones: ['Forward' 'Unknown' 'Center' 'Guard']
Nombres de equipos: ['Trail Blazers' 'Free Agent' 'Lakers' 'Kings' 'Grizzlies' 'Warriors'
 '76ers' 'Pistons' 'Raptors' 'Clippers' 'Knicks' 'Magic' 'Rockets' 'Suns'
 'Nuggets' 'Nets' 'Heat' 'Cavaliers' 'Jazz' 'Mavericks' 'Pelicans'
 'Wizards' 'Spurs' 'Bucks' 'Timberwolves' 'Bulls' 'Pacers' 'Hawks'
 'Celtics' 'Hornets' 'Falcons' 'Thunder' 'Stags' 'Bombers' 'Olympians'
 'Capitols' 'Rebels' 'Blackhawks' 'Bobcats' 'Jets' 'Steamrollers'
 'Redskins' 'Zephyrs']
Abreviaturas de equipos: ['POR' 'FA' 'LAL' 'SAC' 'VAN' 'GSW' 'PHI' 'PIT' 'TOR' 'LAC' 'NYK' 'ORL'
 'SDR' 'PHX' 'DET' 'MEM' 'DEN' 'NJN' 'MIA' 'CLE' 'UTA' 'DAL' 'NOP' 'WAS'
 'BKN' 'SAS' 'MIL' 'MIN' 'CHI' 'SFW' 'IND' 'HOU' 'ATL' 'STL' 'KCK' 'BOS'
 'PHW' 'NOH' 'FTW' 'CIN' 'BAL' 'DEF' 'BLT' 'BUF' 'SEA' 'CHS' 'CHA' 'BOM'
 'INO' 'SDC' 'CLR' 'TCB' 'CHH' 'MIH' 'CHP' 'WAT' 'JET' 'MNL' 'NOK' 'DN'
 'ROC' 'OKC' 'PRO' 'SHE' 'CHZ' 'HUS' 'AND' 'CAP']
C√

In [11]:

# Verificaci√≥n r√°pida de carga
for name, df in dataframes.items():
    print(f"{name}: {df.shape}")

draft: (596, 14)
game: (65698, 55)
game_summary: (58110, 14)
other_stats: (28271, 26)
play_by_play: (13592899, 34)
player: (4831, 5)
common_player_info: (4171, 33)
mvp: (143, 22)
salarios: (1350, 12)
player_master: (4831, 32)


In [12]:
# Remover common_player_info si est√° presente
dataframes.pop('common_player_info', None)

person_id first_name     last_name   display_first_last  \
0        76001       Alaa     Abdelnaby       Alaa Abdelnaby   
1        76003     Kareem  Abdul-Jabbar  Kareem Abdul-Jabbar   
2         1505      Tariq   Abdul-Wahad    Tariq Abdul-Wahad   
3          949    Shareef   Abdur-Rahim  Shareef Abdur-Rahim   
4        76005        Tom     Abernethy        Tom Abernethy   
...        ...        ...           ...                  ...   
4166   1627835       Paul        Zipser          Paul Zipser   
4167   1627790       Ante         Zizic           Ante Zizic   
4168     78647        Jim          Zoet             Jim Zoet   
4169   1627826      Ivica         Zubac          Ivica Zubac   
4170     78650       Matt         Zunic           Matt Zunic   

     display_last_comma_first  display_fi_last          player_slug  \
0             Abdelnaby, Alaa     A. Abdelnaby       alaa-abdelnaby   
1        Abdul-Jabbar, Kareem  K. Abdul-Jabbar  kareem-abdul-jabbar   
2          Abdul-Wahad, Tariq   T. Abdul-Wahad    tariq-abdul-wahad   
3        Abdur-Rahim, Shareef   S. Abdur-Rahim  shareef-abdur-rahim   
4              Abernethy, Tom     T. Abernethy        tom-abernethy   
...                       ...              ...                  ...   
4166             Zipser, Paul        P. Zipser          paul-zipser   
4167              Zizic, Ante         A. Zizic           ante-zizic   
4168                Zoet, Jim          J. Zoet             jim-zoet   
4169             Zubac, Ivica         I. Zubac          ivica-zubac   
4170              Zunic, Matt         M. Zunic           matt-zunic   

                birthdate             school  country  ...  \
0     1968-06-24 00:00:00               Duke      USA  ...   
1     1947-04-16 00:00:00               UCLA      USA  ...   
2     1974-11-03 00:00:00     San Jose State   France  ...   
3     1976-12-11 00:00:00         California      USA  ...   
4     1954-05-06 00:00:00            Indiana      USA  ...   
...                   ...                ...      ...  ...   
4166  1994-02-18 00:00:00      Bayern Munich  Germany  ...   
4167  1997-01-04 00:00:00        Darussafaka  Croatia  ...   
4168  1953-12-30 00:00:00         Kent State      USA  ...   
4169  1997-03-18 00:00:00        Mega Basket  Croatia  ...   
4170  1919-12-19 00:00:00  George Washington      USA  ...   

                       playercode from_year  to_year  dleague_flag nba_flag  \
0          HISTADD_alaa_abdelnaby    1990.0   1994.0             N        Y   
1     HISTADD_kareem_abdul-jabbar    1969.0   1988.0             N        Y   
2               tariq_abdul-wahad    1997.0   2003.0             N        Y   
3             shareef_abdur-rahim    1996.0   2007.0             N        Y   
4           HISTADD_tom_abernethy    1976.0   1980.0             N        Y   
...                           ...       ...      ...           ...      ...   
4166                  paul_zipser    2016.0   2017.0             Y        Y   
4167                   ante_zizic    2017.0   2019.0             Y        Y   
4168             HISTADD_jim_zoet    1982.0   1982.0             N        Y   
4169                  ivica_zubac    2016.0   2023.0             Y        Y   
4170           HISTADD_matt_zunic    1948.0   1948.0             N        Y   

     games_played_flag draft_year draft_round  draft_number greatest_75_flag  
0                    Y       1990           1            25                N  
1                    Y       1969           1             1                Y  
2                    Y       1997           1            11                N  
3                    Y       1996           1             3                N  
4                    Y       1976           3            43                N  
...                ...        ...         ...           ...              ...  
4166                 Y       2016           2            48                N  
4167                 Y       2016           1            

In [13]:
print("\nüìÇ Archivos cargados:")
for name, df in dataframes.items():
    print(f"- {name}: {df.shape}")


üìÇ Archivos cargados:
- draft: (596, 14)
- game: (65698, 55)
- game_summary: (58110, 14)
- other_stats: (28271, 26)
- play_by_play: (13592899, 34)
- player: (4831, 5)
- mvp: (143, 22)
- salarios: (1350, 12)
- player_master: (4831, 32)


## Diagn√≥stico de Nulos y Tipos de Datos (Optimizado)

En esta secci√≥n, se analiza la calidad de los datos en t√©rminos de valores nulos y tipos de datos. Esto es fundamental para identificar problemas que podr√≠an afectar el an√°lisis y el rendimiento del c√≥digo.

---

### Lista de DataFrames Analizados

- Draft: Datos de selecci√≥n de jugadores en el draft.
- Team Info: Informaci√≥n b√°sica sobre los equipos.
- Game: Detalles de cada partido.
- Game Summary: Estad√≠sticas generales por partido.
- Other Stats: M√©tricas avanzadas como puntos en la pintura y rebotes.
- Play-by-Play: Jugadas individuales por partido.
- Player Master: Informaci√≥n combinada de jugadores.
- MVP: Datos de premios al jugador m√°s valioso (MVP).
- Salarios: Informaci√≥n salarial de los jugadores.
- Common Player Info: Detalles personales y biogr√°ficos de los jugadores.

---

### Proceso Aplicado

1. Filtrado de Nulos y Tipos: Se calcula un resumen de nulos y tipos de datos para cada DataFrame.
2. Filtrado Inteligente: Solo se procesan DataFrames que no est√°n vac√≠os para evitar errores.
3. Optimizaci√≥n del Formato: Se incluyen columnas para "Tipo de Dato", "Valores Nulos" y "% de Nulos" para facilitar la limpieza posterior.
4. Estructura Mejorada: Se organizan los resultados en un formato tabular claro para facilitar la interpretaci√≥n.

---

Esta secci√≥n es cr√≠tica para identificar las columnas que requieren limpieza o eliminaci√≥n antes de proceder con los an√°lisis estad√≠sticos y visualizaciones.

In [14]:
#  Lista de DataFrames a Analizar

dataframes = {
    "Draft": draft_filtrado,
    "Game": game_filtrado,
    "Game Summary": game_summary_filtrado,
    "Other Stats": other_stats_filtrado,
    "Play-by-Play": play_by_play_filtrado,
    "Player Master": player_master_filtrado,  
    "MVP": MVP_filtrado,
    "Salarios": salarios_filtrado,
    }

#  Funci√≥n para Diagn√≥stico de Nulos y Tipos de Datos (Optimizada)
def diagnostico_nulos(df, nombre="DataFrame"):
    if df.empty:
        return pd.DataFrame()  # Evitar errores si el DataFrame est√° vac√≠o

    resumen = pd.DataFrame({
        "Tipo de dato": df.dtypes,
        "Valores nulos": df.isnull().sum(),
        "% de nulos": round(df.isnull().mean() * 100, 2),
        "Total Filas": len(df)
    })
    # Filtrar solo columnas con nulos
    resumen = resumen[resumen["Valores nulos"] > 0]
    # Ordenar por porcentaje de nulos
    resumen = resumen.sort_values(by="% de nulos", ascending=False)
    # Agregar nombre del DataFrame como √≠ndice
    resumen.index.name = f"Diagn√≥stico de: {nombre}"
    return resumen

#  Crear un resumen para todos los DataFrames
resumen_total = pd.DataFrame()
for nombre, df in dataframes.items():
    if not df.empty:  # Solo procesar si el DataFrame no est√° vac√≠o
        resumen = diagnostico_nulos(df, nombre)
        resumen_total = pd.concat([resumen_total, resumen])

#  Resetear √≠ndice para facilitar la visualizaci√≥n
resumen_total.reset_index(inplace=True)
resumen_total.index.name = "Fila"

#  Mostrar resumen combinado
from IPython.display import display

display(resumen_total)



index Tipo de dato  Valores nulos  % de nulos  Total Filas
Fila                                                                        
0          organization       object              2        0.34          596
1     organization_type       object              2        0.34          596
2          fg3_pct_home      float64          19074       29.03        65698
3             dreb_home      float64          18999       28.92        65698
4             dreb_away      float64          18998       28.92        65698
...                 ...          ...            ...         ...          ...
100           2022-2023      float64            811       60.07         1350
101           2018-2019      float64            808       59.85         1350
102           2020-2021      float64            800       59.26         1350
103           2017-2018      float64            794       58.81         1350
104           2021-2022      float64            735       54.44         1350

[105 rows x 5 columns]

## Diagn√≥stico de Nulos y Tipos de Datos (Optimizado)

En esta secci√≥n, se analiza la calidad de los datos en t√©rminos de valores nulos y tipos de datos para identificar posibles problemas antes de proceder con el an√°lisis estad√≠stico y las visualizaciones.

---

### DataFrames Incluidos en el An√°lisis:

- **Draft:** Datos de selecci√≥n de jugadores en el draft.
- **Team Info:** Informaci√≥n b√°sica sobre los equipos.
- **Game:** Detalles de cada partido.
- **Game Summary:** Estad√≠sticas generales por partido.
- **Other Stats:** M√©tricas avanzadas como puntos en la pintura y rebotes.
- **Play-by-Play:** Jugadas individuales por partido.
- **Player Master:** Informaci√≥n combinada de jugadores, incluyendo datos personales y de desempe√±o.
- **Common Player Info:** Detalles personales y biogr√°ficos de los jugadores.
- **Salarios:** Informaci√≥n salarial de los jugadores, incluyendo datos hist√≥ricos por temporada.

---

### Proceso Aplicado:

1. **Filtrado de Nulos y Tipos:** Se calcula un resumen de nulos y tipos de datos para cada DataFrame.
2. **Filtrado Inteligente:** Solo se procesan DataFrames que no est√°n vac√≠os para evitar errores.
3. **Optimizaci√≥n del Formato:** Se incluyen columnas para "Tipo de Dato", "Valores Nulos" y "% de Nulos" para facilitar la limpieza posterior.
4. **Estructura Mejorada:** Se organizan los resultados en un formato tabular claro para facilitar la interpretaci√≥n.

---

Esta lista es esencial para estructurar correctamente el an√°lisis de nulos y tipos de datos antes de proceder con las limpiezas y uniones m√°s avanzadas.



In [ ]:

#  Funci√≥n para Diagn√≥stico de Nulos y Tipos de Datos (Optimizada)
def diagnostico_nulos(df, nombre="DataFrame"):
    if df.empty:
        return pd.DataFrame()  # Evitar errores si el DataFrame est√° vac√≠o

    # Crear resumen con tipos de datos y nulos
    resumen = pd.DataFrame({
        "Tipo de dato": df.dtypes,
        "Valores nulos": df.isnull().sum(),
        "% de nulos": round(df.isnull().mean() * 100, 2)
    })
    
    # Filtrar solo columnas con nulos
    resumen = resumen[resumen["Valores nulos"] > 0]
    
    # Ordenar por porcentaje de nulos
    resumen = resumen.sort_values(by="% de nulos", ascending=False)
    
    # A√±adir nombre del DataFrame para facilitar el an√°lisis
    resumen.insert(0, "DataFrame", nombre)
    
    return resumen

#  Lista de DataFrames a analizar
dataframes = {
    "Draft":  draft_filtrado,
    "Team Info": team_info_filtrado,
    "Game": game_filtrado,
    "Game Summary": game_summary_filtrado,
    "Other Stats": other_stats_filtrado,
    "Play-by-Play": play_by_play_filtrado,
    "Player Master": player_master_filtrado, 
    "Salarios": salarios_filtrado
}

#  Crear resumen combinado
resumen_total = pd.concat(
    [diagnostico_nulos(df, nombre) for nombre, df in dataframes.items() if not df.empty],
    ignore_index=True
)

# Mostrar resumen combinado en Jupyter
from IPython.display import display

# Mostrar el resumen de nulos y tipos
display(resumen_total)



## Manejo de Valores Nulos en Datos Salariales

En esta secci√≥n, se reemplazan los valores nulos en las columnas de salarios para evitar sesgos en el an√°lisis posterior. Se utilizan dos pasos para asegurar que los datos sean consistentes:

---

### Pasos Aplicados:

1. **Relleno Inicial con Ceros:**
   - Los valores nulos se rellenan con ceros para evitar errores en c√°lculos posteriores.
   - Esto asume que los nulos representan temporadas sin salario registrado.

```python
# Rellenar primero con ceros para temporadas sin datos
salarios_filtrado.iloc[:, 2:] = salarios_filtrado.iloc[:, 2:].fillna(0)
```

2. **Reemplazo de Ceros con Promedio del Jugador:**
   - Luego, los ceros se reemplazan por el salario promedio del jugador para evitar distorsiones en los an√°lisis.
   - Los datos se convierten a **`Int64`** para mantener consistencia en los tipos de datos.

```python
# Reemplazar ceros con promedio y convertir a Int64
salarios_filtrado.iloc[:, 2:] = salarios_filtrado.iloc[:, 2:].apply(
    lambda x: x.replace(0, int(x.mean())) if x.mean() > 0 else x, axis=1
)
```

---

Estos pasos aseguran que los datos salariales est√©n completos y listos para an√°lisis avanzados sin introducir sesgos significativos.

In [ ]:
# Rellenar primero con ceros para temporadas sin datos
salarios_filtrado.iloc[:, 2:] = salarios_filtrado.iloc[:, 2:].fillna(0)

# Reemplazar ceros con promedio y convertir a Int64
salarios_filtrado.iloc[:, 2:] = salarios_filtrado.iloc[:, 2:].apply(
    lambda x: x.replace(0, int(x.mean())) if x.mean() > 0 else x, axis=1
)


In [ ]:
print(salarios_filtrado.describe())

                 id     2013-2014     2014-2015     2015-2016     2016-2017  \
count  1.350000e+03  1.350000e+03  1.350000e+03  1.350000e+03  1.350000e+03   
mean   9.395236e+05  2.317729e+06  2.206414e+06  2.201913e+06  2.566439e+06   
std    7.362326e+05  3.779237e+06  3.667483e+06  3.836135e+06  4.707570e+06   
min    1.990000e+02  5.840000e+02  5.840000e+02  5.840000e+02  5.840000e+02   
25%    2.023892e+05  1.310118e+05  1.118780e+05  1.017780e+05  1.066895e+05   
50%    1.626196e+06  7.222430e+05  7.110375e+05  6.241105e+05  5.969765e+05   
75%    1.629618e+06  2.795006e+06  2.500000e+06  2.275785e+06  2.435550e+06   
max    1.631466e+06  3.045300e+07  2.350000e+07  2.500000e+07  3.096345e+07   

          2017-2018     2018-2019     2019-2020     2020-2021     2021-2022  \
count  1.350000e+03  1.350000e+03  1.350000e+03  1.350000e+03  1.350000e+03   
mean   2.758026e+06  2.913838e+06  3.028892e+06  3.267242e+06  3.473654e+06   
std    5.352870e+06  5.768427e+06  6.322484e+06  6.

## Eliminaci√≥n de Columnas con Muchos Nulos

El siguiente bloque de c√≥digo elimina columnas con m√°s del **60%** de valores nulos en varios **DataFrames** utilizados en el an√°lisis de desempe√±o de los **Detroit Pistons**. Esta limpieza es fundamental para reducir el sesgo y mejorar la calidad de los datos antes de la integraci√≥n en SQL y otros an√°lisis.

---

### üìã **Funci√≥n `eliminar_columnas_con_muchos_nulos()`**

Esta funci√≥n elimina columnas con m√°s del **60%** de nulos en cualquier DataFrame que se le pase como par√°metro.

```python
# Funci√≥n para eliminar columnas con muchos nulos
def eliminar_columnas_con_muchos_nulos(df, umbral=0.6, nombre="DataFrame"):
    # Calcular el porcentaje de nulos
    umbral_nulos = df.isnull().mean() > umbral
    # Obtener los nombres de las columnas a eliminar
    columnas_a_eliminar = umbral_nulos[umbral_nulos].index.tolist()
    
    # Eliminar columnas
    df_limpio = df.drop(columns=columnas_a_eliminar)
    
    # Mostrar resumen del proceso
    if columnas_a_eliminar:
        print(f" Eliminadas {len(columnas_a_eliminar)} columnas de '{nombre}':")
        print(columnas_a_eliminar)
    else:
        print(f" Ninguna columna eliminada en '{nombre}' (umbral {int(umbral * 100)}%)")
    
    return df_limpio


In [ ]:
# Funci√≥n para eliminar columnas con muchos nulos
def eliminar_columnas_con_muchos_nulos(df, umbral=0.6, nombre="DataFrame"):
    # Calcular el porcentaje de nulos
    umbral_nulos = df.isnull().mean() > umbral
    # Obtener los nombres de las columnas a eliminar
    columnas_a_eliminar = umbral_nulos[umbral_nulos].index.tolist()
    
    # Eliminar columnas
    df_limpio = df.drop(columns=columnas_a_eliminar)
    
    # Mostrar resumen del proceso
    if columnas_a_eliminar:
        print(f" Eliminadas {len(columnas_a_eliminar)} columnas de '{nombre}':")
        print(columnas_a_eliminar)
    else:
        print(f" Ninguna columna eliminada en '{nombre}' (umbral {int(umbral * 100)}%)")
    
    return df_limpio

# Aplicar limpieza a cada dataset
draft_filtrado = eliminar_columnas_con_muchos_nulos(draft_filtrado, nombre="Draft")
team_info_filtrado = eliminar_columnas_con_muchos_nulos(team_info_filtrado, nombre="Team Info")
game_filtrado = eliminar_columnas_con_muchos_nulos(game_filtrado, nombre="Game")
game_summary_filtrado = eliminar_columnas_con_muchos_nulos(game_summary_filtrado, nombre="Game Summary")
other_stats_filtrado = eliminar_columnas_con_muchos_nulos(other_stats_filtrado, nombre="Other Stats")
play_by_play_filtrado = eliminar_columnas_con_muchos_nulos(play_by_play_filtrado, nombre="Play-by-Play")
player_master_filtrado= eliminar_columnas_con_muchos_nulos(player_master_filtrado, nombre="Player")
common_info_filtrado = eliminar_columnas_con_muchos_nulos(common_info_filtrado, nombre="Common Player Info")
MVP_filtrado = eliminar_columnas_con_muchos_nulos(MVP_filtrado, nombre="MVP")
Salarios_filtrado = eliminar_columnas_con_muchos_nulos(salarios_filtrado, nombre="Salarios")


## An√°lisis de Columnas con Valores Nulos

En esta secci√≥n, se identifican las columnas con valores nulos en cada uno de los DataFrames filtrados. Esto es fundamental para evaluar la calidad de los datos antes de proceder con an√°lisis m√°s avanzados.

---

### Funci√≥n Utilizada:

La funci√≥n **`columnas_con_nulos()`** calcula y muestra el n√∫mero y porcentaje de valores nulos para cada columna de un DataFrame dado. Solo muestra las columnas que efectivamente tienen nulos para evitar salidas innecesarias.

```python
# Funci√≥n para mostrar columnas con nulos en cada DataFrame
def columnas_con_nulos(nombre, df):
    # Crear resumen de nulos
    nulos = df.isnull().sum()
    total_filas = len(df)
    porcentaje_nulos = (nulos / total_filas * 100).round(2)
    
    # Filtrar solo columnas con nulos
    columnas_con_nulos = nulos[nulos > 0]
    porcentajes = porcentaje_nulos[nulos > 0]
    
    # Crear tabla de resumen
    resumen = pd.DataFrame({
        "Valores nulos": columnas_con_nulos,
        "% de nulos": porcentajes
    }).sort_values(by="% de nulos", ascending=False)
    
    # Mostrar solo si hay columnas con nulos
    if not resumen.empty:
        print(f"\nDataset: {nombre}")
        display(resumen)
```

---

### DataFrames Analizados:

- **Draft:** Datos de selecci√≥n de jugadores en el draft.
- **Team Info:** Informaci√≥n b√°sica sobre los equipos.
- **Game:** Detalles de cada partido.
- **Game Summary:** Estad√≠sticas generales por partido.
- **Other Stats:** M√©tricas avanzadas como puntos en la pintura y rebotes.
- **Play-by-Play:** Jugadas individuales por partido.
- **Player Master:** Informaci√≥n combinada de jugadores, incluyendo datos personales y de desempe√±o.
- **MVP:** Datos de premios al jugador m√°s valioso (MVP).
- **Salarios:** Informaci√≥n salarial de los jugadores.

---

### Resultado Esperado:

El c√≥digo muestra un resumen claro para cada DataFrame que contiene columnas con valores nulos, ordenando los resultados por porcentaje de nulos de mayor a menor.


In [ ]:
# Funci√≥n para mostrar columnas con nulos en cada DataFrame
def columnas_con_nulos(nombre, df):
    # Crear resumen de nulos
    nulos = df.isnull().sum()
    total_filas = len(df)
    porcentaje_nulos = (nulos / total_filas * 100).round(2)
    
    # Filtrar solo columnas con nulos
    columnas_con_nulos = nulos[nulos > 0]
    porcentajes = porcentaje_nulos[nulos > 0]
    
    # Crear tabla de resumen
    resumen = pd.DataFrame({
        "Valores nulos": columnas_con_nulos,
        "% de nulos": porcentajes
    }).sort_values(by="% de nulos", ascending=False)
    
    # Mostrar solo si hay columnas con nulos
    if not resumen.empty:
        print(f"\nDataset: {nombre}")
        display(resumen)

# Aplicar a todos los DataFrames filtrados
dataframes_filtrados = {
    "Draft": draft_filtrado,
    "Team Info": team_info_filtrado,
    "Game": game_filtrado,
    "Game Summary": game_summary_filtrado,
    "Other Stats": other_stats_filtrado,
    "Play-by-Play": play_by_play_filtrado,
    "Player Master": player_master_filtrado,
    "MVP": MVP_filtrado,
    "Salarios": salarios_filtrado  
}

# Mostrar resultados
for nombre, df in dataframes_filtrados.items():
    if not df.empty:
        columnas_con_nulos(nombre, df)




NameError: name 'team_info_filtrado' is not defined

##  An√°lisis de Columnas con Nulos en M√∫ltiples DataFrames

Este bloque de c√≥digo identifica las columnas con valores nulos en cada uno de los **DataFrames** filtrados para el an√°lisis del desempe√±o de los **Detroit Pistons**. Esto es crucial para decidir qu√© columnas deben ser limpiadas, rellenadas o eliminadas antes de exportar los datos a SQL.

---

### *DataFrames Analizados:

- **Draft:** Historial de selecci√≥n de jugadores.
- **Team Info:** Informaci√≥n general de equipos.
- **Game:** Detalles de partidos.
- **Game Summary:** Resumen estad√≠stico por partido.
- **Other Stats:** M√©tricas avanzadas como rebotes y p√©rdidas.
- **Play-by-Play:** Jugadas individuales.
- **Player:** Identificadores y nombres de jugadores.
- **Common Player Info:** Informaci√≥n personal de jugadores.
- **MVP:** Jugadores m√°s valiosos por temporada.
- **Salarios:** Informaci√≥n salarial de jugadores.

---

###  Funci√≥n para Mostrar Columnas con Nulos

La funci√≥n **`columnas_con_nulos()`** genera un resumen de las columnas que contienen valores nulos en cada **DataFrame**:

- **Valores nulos:** N√∫mero total de celdas vac√≠as.
- **% de nulos:** Porcentaje de celdas vac√≠as respecto al total de filas.

```python
# Funci√≥n para mostrar columnas con nulos en cada DataFrame
def columnas_con_nulos(nombre, df):
    # Crear resumen de nulos
    nulos = df.isnull().sum()
    total_filas = len(df)
    porcentaje_nulos = (nulos / total_filas * 100).round(2)
    
    # Filtrar solo columnas con nulos
    columnas_con_nulos = nulos[nulos > 0]
    porcentajes = porcentaje_nulos[nulos > 0]
    
    # Crear tabla de resumen
    resumen = pd.DataFrame({
        "Valores nulos": columnas_con_nulos,
        "% de nulos": porcentajes
    }).sort_values(by="% de nulos", ascending=False)
    
    # Mostrar solo si hay columnas con nulos
    if not resumen.empty:
        print(f"\n Dataset: {nombre}")
        print(resumen)


In [ ]:
 
# Funci√≥n para crear un resumen de nulos por DataFrame 
def resumen_nulos_por_dataset(nombre_df, df):
    if df.empty:
        return pd.DataFrame()  # Evita errores si el DataFrame est√° vac√≠o
    
    # Crear resumen de nulos
    resumen = pd.DataFrame({
        "Columna": df.columns,
        "% de nulos": df.isnull().mean().round(4) * 100
    })
    
    # Filtrar solo columnas con nulos
    resumen = resumen[resumen["% de nulos"] > 0].sort_values(by="% de nulos", ascending=False)
    
    # A√±adir nombre del DataFrame
    resumen.insert(0, "Dataset", nombre_df)
    
    return resumen

# Lista de DataFrames filtrados (Corregido)
dataframes_filtrados = {
    "Draft": draft_filtrado,
    "Team Info": team_info_filtrado,
    "Game": game_filtrado,
    "Game Summary": game_summary_filtrado,
    "Other Stats": other_stats_filtrado,
    "Play-by-Play": play_by_play_filtrado,
    "Player Master": player_master_filtrado,
    "MVP": MVP_filtrado,
    "Salarios": salarios_filtrado  
}

# Crear resumen combinado 
resumen_total_nulos = pd.concat(
    [resumen_nulos_por_dataset(nombre, df) for nombre, df in dataframes_filtrados.items() if not df.empty],
    ignore_index=True
)

# Mostrar resumen completo
pd.set_option('display.max_rows', 100)
display(resumen_total_nulos)



NameError: name 'team_info_filtrado' is not defined

In [ ]:
# Mostrar todos los DataFrames cargados en memoria
%whos DataFrame

In [ ]:
# Verificar que la fusi√≥n es correcta
print(player_master_df.info())
print(player_master_df.head())

NameError: name 'player_master_df' is not defined

In [ ]:
# Revisar las columnas de player_master_filtrado
print("Player Master Columns:")
print(player_master_filtrado.columns)

# Revisar las columnas de common_info_filtrado (si existe)
try:
    print("\nCommon Player Info Columns:")
    print(common_info_filtrado.columns)
except NameError:
    print("\nCommon Player Info no est√° definido.")

Player Master Columns:
Index(['person_id', 'full_name', 'first_name_player', 'last_name_player',
       'is_active', 'player_slug', 'birthdate', 'school', 'country',
       'last_affiliation', 'height', 'weight', 'season_exp', 'jersey',
       'position', 'rosterstatus', 'games_played_current_season_flag',
       'team_id', 'team_name', 'team_abbreviation', 'team_code', 'team_city',
       'playercode', 'from_year', 'to_year', 'dleague_flag', 'nba_flag',
       'games_played_flag', 'draft_year', 'draft_round', 'draft_number',
       'greatest_75_flag'],
      dtype='object')

Common Player Info Columns:
Index(['person_id', 'first_name', 'last_name', 'display_first_last',
       'display_last_comma_first', 'display_fi_last', 'player_slug',
       'birthdate', 'school', 'country', 'last_affiliation', 'height',
       'weight', 'season_exp', 'jersey', 'position', 'rosterstatus',
       'games_played_current_season_flag', 'team_id', 'team_name',
       'team_abbreviation', 'team_code', 'te

In [ ]:
print(play_by_play_filtrado['homedescription'].value_counts(dropna=True).head(20))
print(play_by_play_filtrado['visitordescription'].value_counts(dropna=True).head(20))

homedescription
LAKERS Rebound           9778
ROCKETS Rebound          9274
NUGGETS Rebound          9263
CLIPPERS Rebound         8939
CAVALIERS Rebound        8858
JAZZ Rebound             8848
HEAT Rebound             8846
MAGIC Rebound            8794
CELTICS Rebound          8749
76ERS Rebound            8659
NETS Rebound             8530
KINGS Rebound            8497
TRAIL BLAZERS Rebound    8483
PISTONS Rebound          8467
SPURS Rebound            8416
RAPTORS Rebound          8414
TIMBERWOLVES Rebound     8384
BULLS Rebound            8369
HAWKS Rebound            8285
GRIZZLIES Rebound        8204
Name: count, dtype: int64
visitordescription
Lakers Rebound           9150
Heat Rebound             8982
Jazz Rebound             8935
Rockets Rebound          8822
Celtics Rebound          8801
Grizzlies Rebound        8729
Nuggets Rebound          8728
76ers Rebound            8659
Spurs Rebound            8617
Clippers Rebound         8616
Magic Rebound            8597
Warriors 

## Limpieza de columnas con alta proporci√≥n de nulos: `homedescription` y `visitordescription`

En este an√°lisis, detectamos que las columnas `homedescription` y `visitordescription` del dataset `play_by_play` tienen aproximadamente un 48% de valores nulos. Estas columnas contienen descripciones textuales de las jugadas, diferenciadas por equipo local y visitante.

Dado que estas columnas pueden aportar contexto en an√°lisis de jugadas espec√≠ficas (rebotes, p√©rdidas, asistencias, etc.), decidimos **no eliminarlas**.

### Estrategia de limpieza: imputaci√≥n basada en contexto

En lugar de rellenar con un valor √∫nico o eliminar filas, optamos por una estrategia m√°s robusta:

- Agrupamos por `team_id` o `player1_id`, para encontrar el valor m√°s com√∫n (`mode`) dentro del contexto de cada equipo o jugador.
- Rellenamos los valores nulos usando ese valor m√°s frecuente para mantener la coherencia contextual.
- Esta t√©cnica ayuda a preservar patrones sem√°nticos que podr√≠an ser √∫tiles para an√°lisis de eventos o modelos descriptivos posteriores.

Este enfoque balancea la conservaci√≥n de informaci√≥n y la reducci√≥n de sesgo por imputaci√≥n gen√©rica.


In [ ]:
# Crear indicadores binarios de acci√≥n
play_by_play_filtrado['home_action_flag'] = play_by_play_filtrado['homedescription'].notnull().astype(int)
play_by_play_filtrado['visitor_action_flag'] = play_by_play_filtrado['visitordescription'].notnull().astype(int)

# Verificar resultado
print(play_by_play_filtrado[['homedescription', 'home_action_flag']].head(10))
print(play_by_play_filtrado[['visitordescription', 'visitor_action_flag']].head(10))

                               homedescription  home_action_flag
0                                          NaN                 0
1  Jump Ball O'Neal vs. Kleine: Tip to Cassell                 1
2                                          NaN                 0
3                 O'Neal REBOUND (Off:0 Def:1)                 1
4              MISS Ceballos 26' 3PT Jump Shot                 1
5                                          NaN                 0
6                      Van Exel P.FOUL (P1.T1)                 1
7                                          NaN                 0
8                       MISS Ceballos 1' Layup                 1
9                               LAKERS Rebound                 1
                  visitordescription  visitor_action_flag
0                                NaN                    0
1                                NaN                    0
2         MISS Cassell 15' Jump Shot                    1
3                                NaN                 

## Limpieza de columnas homedescription y visitordescription

En el dataset `play_by_play`, las columnas `homedescription` y `visitordescription` contienen informaci√≥n textual sobre las acciones realizadas por los equipos local y visitante, respectivamente. Sin embargo, estas columnas pueden contener valores nulos debido a que no siempre hay una acci√≥n relevante por cada equipo en cada fila.

Para evitar eliminar o imputar incorrectamente estos valores nulos, se utilizan las columnas auxiliares `home_action_flag` y `visitor_action_flag`. Estas indican si la acci√≥n de una fila corresponde al equipo local (`home_action_flag == 1`) o al visitante (`visitor_action_flag == 1`).

### Criterio aplicado:
- Si `home_action_flag` es 1 y `homedescription` est√° nulo, se considera un **nulo real** y se reemplaza por `"Desconocido"`.
- Si `visitor_action_flag` es 1 y `visitordescription` est√° nulo, tambi√©n se reemplaza por `"Desconocido"`.

Este tratamiento conserva la l√≥gica del juego y evita eliminar datos valiosos sin justificaci√≥n.

Se puede modificar `"Desconocido"` por otro valor est√°ndar si el an√°lisis lo requiere.


In [ ]:
# --- Limpiar homedescription y visitordescription respetando los flags ---

# Si home_action_flag == 1 y homedescription est√° nulo, es un nulo real
nulos_home = (
    (play_by_play_filtrado['home_action_flag'] == 1) &
    (play_by_play_filtrado['homedescription'].isnull())
)

# Si visitor_action_flag == 1 y visitordescription est√° nulo, es un nulo real
nulos_visitor = (
    (play_by_play_filtrado['visitor_action_flag'] == 1) &
    (play_by_play_filtrado['visitordescription'].isnull())
)

# Mostrar cu√°ntos nulos reales hay
print(f"Nulos reales en homedescription: {nulos_home.sum()}")
print(f"Nulos reales en visitordescription: {nulos_visitor.sum()}")

# Reemplazar esos nulos reales por la palabra 'Desconocido' o lo que prefieras
play_by_play_filtrado.loc[nulos_home, 'homedescription'] = 'Desconocido'
play_by_play_filtrado.loc[nulos_visitor, 'visitordescription'] = 'Desconocido'

Nulos reales en homedescription: 0
Nulos reales en visitordescription: 0


In [ ]:
columnas_objetivo = [
    "person1type", "player1_name", "player1_team_id",
    "player1_team_city", "player1_team_nickname", "player1_team_abbreviation"
]

# Calcular y mostrar el porcentaje de nulos
porcentaje_nulos_ppb_objetivo = (
    play_by_play_filtrado[columnas_objetivo]
    .isnull()
    .mean()
    .round(4) * 100
).sort_values(ascending=False)

print(porcentaje_nulos_ppb_objetivo)


player1_team_city            8.94
player1_team_id              8.94
player1_team_nickname        8.94
player1_team_abbreviation    8.94
player1_name                 8.89
person1type                  0.02
dtype: float64


## Imputaci√≥n de valores nulos mediante la moda

En el dataset `play_by_play_filtrado` se identificaron varias columnas con valores nulos significativos, especialmente relacionadas con el jugador principal (`player1`) y las descripciones de jugadas.

### Criterio aplicado:
Se eligi√≥ imputar los valores nulos usando la **moda** (el valor m√°s frecuente) porque:

- Es una t√©cnica robusta para columnas categ√≥ricas o de texto corto.
- No distorsiona la distribuci√≥n de los datos cuando hay categor√≠as dominantes.
- Permite conservar el volumen de datos sin eliminar registros completos.

### Columnas tratadas:
- `visitordescription`: descripci√≥n de jugada del equipo visitante.
- `homedescription`: descripci√≥n de jugada del equipo local.
- `player1_team_city`: ciudad del equipo del jugador 1.
- `player1_team_id`: identificador del equipo del jugador 1.
- `player1_team_nickname`: apodo del equipo del jugador 1.
- `player1_team_abbreviation`: abreviatura del equipo del jugador 1.
- `player1_name`: nombre del jugador principal de la jugada.
- `person1type`: tipo de participaci√≥n del jugador (rol en la jugada), con solo ~0.05% de nulos.

### Resultado:
Todas las columnas fueron imputadas exitosamente con sus respectivas modas, permitiendo una base de datos m√°s limpia y lista para an√°lisis exploratorio, modelado o carga en bases relacionales.



In [ ]:
# Lista de columnas a rellenar por moda
columnas_moda = [
    "visitordescription",
    "homedescription",
    "player1_team_city",
    "player1_team_id",
    "player1_team_nickname",
    "player1_team_abbreviation",
    "player1_name",
    "person1type"  # agregado para completar limpieza
]

# Rellenar por moda en cada columna
for col in columnas_moda:
    if col in play_by_play_filtrado.columns:
        moda = play_by_play_filtrado[col].mode(dropna=True)
        if not moda.empty:
            valor_moda = moda[0]
            play_by_play_filtrado[col].fillna(valor_moda, inplace=True)
            print(f"‚úÖ Columna '{col}' rellenada con la moda: {valor_moda}")
        else:
            print(f"‚ö†Ô∏è No se pudo calcular la moda de la columna '{col}' (vac√≠a o solo nulos).")
    else:
        print(f"‚ö†Ô∏è La columna '{col}' no existe en el DataFrame.")




C:\Users\admin\AppData\Local\Temp\ipykernel_19072\634657652.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  play_by_play_filtrado[col].fillna(valor_moda, inplace=True)


‚úÖ Columna 'visitordescription' rellenada con la moda: Lakers Rebound
‚úÖ Columna 'homedescription' rellenada con la moda: LAKERS Rebound
‚úÖ Columna 'player1_team_city' rellenada con la moda: Los Angeles
‚úÖ Columna 'player1_team_id' rellenada con la moda: 1610612747.0
‚úÖ Columna 'player1_team_nickname' rellenada con la moda: Lakers
‚úÖ Columna 'player1_team_abbreviation' rellenada con la moda: LAL
‚úÖ Columna 'player1_name' rellenada con la moda: LeBron James
‚úÖ Columna 'person1type' rellenada con la moda: 4.0


## Resumen general de limpieza de datos

Durante el preprocesamiento de los datasets del proyecto NBA, se aplicaron las siguientes estrategias para asegurar una base de datos limpia y lista para an√°lisis y modelado:

---

### ‚úÖ 1. Eliminaci√≥n de columnas con m√°s del 60% de valores nulos
Se implement√≥ una funci√≥n `eliminar_columnas_con_muchos_nulos()` con un umbral del 60%, eliminando aquellas columnas con informaci√≥n insuficiente o poco relevante debido a su alta tasa de vac√≠os.

---

### ‚úÖ 2. Imputaci√≥n de columnas con pocos valores nulos usando la **moda**
Para columnas que conten√≠an una proporci√≥n baja o moderada de nulos (entre ellas `player1_name`, `player1_team_id`, `person1type`, etc.), se utiliz√≥ el valor m√°s frecuente (moda) como estrategia de imputaci√≥n.

**¬øPor qu√© se eligi√≥ la moda?**
- Es una t√©cnica eficaz para columnas categ√≥ricas o de texto corto.
- Mantiene la coherencia sem√°ntica de los datos sin distorsionar la distribuci√≥n.
- Permite conservar registros completos sin necesidad de eliminarlos.
- Es especialmente √∫til cuando hay valores dominantes que representan comportamientos frecuentes y relevantes.

---

### ‚úÖ 3. Limpieza contextual especializada: `homedescription` y `visitordescription`
Estas columnas conten√≠an descripciones clave de jugadas del equipo local y visitante, pero presentaban aproximadamente un 48% de valores nulos.

- Se utilizaron las columnas auxiliares `home_action_flag` y `visitor_action_flag` para detectar cu√°ndo el nulo era realmente relevante.
- Se imputaron con el valor `"Desconocido"` √∫nicamente cuando la acci√≥n estaba marcada como existente (`flag == 1`).
- Este enfoque preserva la estructura y la l√≥gica de juego sin imputar en exceso ni eliminar registros.

---

### üîç 4. Validaci√≥n final
Se verific√≥ que no quedaran valores nulos en columnas relevantes tras la limpieza. Se confirm√≥ con:

```python
play_by_play_filtrado.isnull().sum().sort_values(ascending=False).head(15)


In [ ]:
play_by_play_filtrado.isnull().sum().sort_values(ascending=False).head(15)

## Mapa de calor: Correlaci√≥n de m√©tricas de desempe√±o por partido

Este an√°lisis busca identificar relaciones clave entre diferentes indicadores de rendimiento colectivo a nivel de partido. Se utilizaron variables del dataset `other_stats_filtrado`, que reflejan aspectos estrat√©gicos del juego tanto para el equipo local como visitante.

### Variables consideradas
- Puntos en la pintura (`pts_paint`)
- Puntos de segunda oportunidad (`pts_2nd_chance`)
- Puntos en contraataque (`pts_fb`)
- Mayor ventaja lograda en el partido (`largest_lead`)
- P√©rdidas de bal√≥n (`team_turnovers`, `total_turnovers`)
- Rebotes colectivos (`team_rebounds`)
- Puntos tras p√©rdidas del rival (`pts_off_to`)

Cada una de estas variables fue separada en sus versiones `*_home` (equipo local) y `*_away` (visitante), para capturar din√°micas en ambos lados del juego.

### Objetivo del mapa de calor
- Identificar **relaciones fuertes** entre m√©tricas que puedan ayudar a explicar el rendimiento de un equipo.
- Detectar **patrones redundantes** que puedan consolidarse en indicadores compuestos.
- Apoyar el dise√±o de futuros modelos predictivos y dashboards de an√°lisis.

### Interpretaci√≥n
- Las correlaciones cercanas a **1 o -1** indican una relaci√≥n lineal fuerte.
- Los valores cercanos a **0** indican poca o nula relaci√≥n.
- Se aplic√≥ una **m√°scara triangular superior** para facilitar la lectura.

Este mapa es una herramienta √∫til para definir qu√© m√©tricas deben priorizarse en el an√°lisis del rendimiento de los Detroit Pistons y para contrastar con su desempe√±o a lo largo del tiempo.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Columnas relevantes del DataFrame other_stats_filtrado
columnas_desempeno = [
    'pts_paint_home', 'pts_2nd_chance_home', 'pts_fb_home', 'largest_lead_home',
    'team_turnovers_home', 'total_turnovers_home', 'team_rebounds_home', 'pts_off_to_home',
    'pts_paint_away', 'pts_2nd_chance_away', 'pts_fb_away', 'largest_lead_away',
    'team_turnovers_away', 'total_turnovers_away', 'team_rebounds_away', 'pts_off_to_away'
]

# Filtrar y calcular la matriz de correlaci√≥n
df_corr = other_stats_filtrado[columnas_desempeno].dropna()
matriz_corr = df_corr.corr()

# M√°scara triangular para mostrar un solo lado del heatmap
mascara = np.triu(np.ones_like(matriz_corr, dtype=bool))

# Generar heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(
    matriz_corr,
    mask=mascara,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    linewidths=0.5,
    square=True,
    cbar_kws={"shrink": .8}
)
plt.title("Mapa de calor - Correlaci√≥n de m√©tricas de desempe√±o por partido")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd

# 1. Asegurar formato de fecha en game_df
game_df['game_date'] = pd.to_datetime(game_df['game_date'], errors='coerce')

# 2. Crear columna 'season' a partir del a√±o de la fecha del partido
game_df['season'] = game_df['game_date'].dt.year

# 3. Eliminar 'season' de other_stats_filtrado si ya existe (evita error en el merge)
if 'season' in other_stats_filtrado.columns:
    other_stats_filtrado = other_stats_filtrado.drop(columns=['season'])

# 4. Realizar el merge para agregar la columna 'season' a other_stats_filtrado
other_stats_filtrado = other_stats_filtrado.merge(
    game_df[['game_id', 'season']],
    on='game_id',
    how='left'
)

# 5. Eliminar filas donde no se pudo asignar una temporada (por seguridad)
other_stats_filtrado = other_stats_filtrado.dropna(subset=['season'])

# 6. Convertir la columna a tipo entero
other_stats_filtrado['season'] = other_stats_filtrado['season'].astype(int)


### An√°lisis de correlaci√≥n: desempe√±o del equipo local

En esta secci√≥n se gener√≥ un **mapa de calor** para identificar relaciones significativas entre variables clave del rendimiento del equipo local durante los partidos. Las m√©tricas provienen del archivo `other_stats.csv` y se analizaron en su forma num√©rica.

**Variables consideradas:**

- `pts_paint_home`: Puntos en la pintura.
- `pts_2nd_chance_home`: Puntos de segunda oportunidad.
- `pts_fb_home`: Puntos por contraataque (fast break).
- `largest_lead_home`: Mayor ventaja alcanzada.
- `team_turnovers_home`: P√©rdidas de bal√≥n del equipo.
- `total_turnovers_home`: P√©rdidas totales.
- `team_rebounds_home`: Rebotes del equipo.
- `pts_off_to_home`: Puntos tras p√©rdidas del rival.

Estas variables fueron seleccionadas por su v√≠nculo directo con el **desempe√±o ofensivo y defensivo** del equipo.

**Metodolog√≠a aplicada:**
- Se eliminaron filas con valores nulos para evitar distorsiones.
- Se calcul√≥ la matriz de correlaci√≥n utilizando `.corr()`.
- Se aplic√≥ una m√°scara para mostrar √∫nicamente la mitad inferior del mapa de calor, facilitando la lectura.
- Se us√≥ un mapa de color `coolwarm` para resaltar relaciones positivas y negativas.

**Objetivo:** Detectar posibles patrones de influencia entre variables, como por ejemplo:
- Mayor n√∫mero de rebotes podr√≠a correlacionar con m√°s puntos en segunda oportunidad.
- Muchas p√©rdidas pueden limitar la capacidad ofensiva reflejada en la ventaja m√°xima.

Este an√°lisis visual constituye un primer paso para dirigir futuras investigaciones o hip√≥tesis sobre el rendimiento del equipo.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Seleccionar columnas num√©ricas relevantes para desempe√±o
columnas_desempeno = [
    'pts_paint_home', 'pts_2nd_chance_home', 'pts_fb_home',
    'largest_lead_home', 'team_turnovers_home', 'total_turnovers_home',
    'team_rebounds_home', 'pts_off_to_home'
]

# 2. Eliminar filas con valores nulos en esas columnas
df = other_stats_filtrado[columnas_desempeno].dropna()

# 3. Calcular la matriz de correlaci√≥n
correlacion = df.corr()

# 4. Generar una m√°scara para ocultar la mitad superior del heatmap
mask = np.triu(np.ones_like(correlacion, dtype=bool))

# 5. Visualizar el mapa de calor
plt.figure(figsize=(10, 8))
sns.heatmap(correlacion, mask=mask, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
plt.title("Mapa de calor: Correlaci√≥n entre variables de desempe√±o del equipo local")
plt.tight_layout()
plt.show()

In [ ]:
# Diccionario con todos los DataFrames filtrados
dataframes_filtrados = {
    "draft": draft_filtrado,
    "team_info": team_info_filtrado,
    "game": game_filtrado,
    "game_summary": game_summary_filtrado,
    "other_stats": other_stats_filtrado,
    "play_by_play": play_by_play_filtrado,
    "player": player_filtrado,
    "common_player_info": common_info_filtrado,
    "MVP": MVP_filtrado,
    "Salarios": Salarios_filtrado
}

# Funci√≥n para verificar duplicados por DataFrame
def revisar_duplicados(df, nombre):
    total = df.shape[0]
    duplicados = df.duplicated().sum()
    porcentaje = round((duplicados / total) * 100, 2) if total > 0 else 0
    print(f"- {nombre}: {duplicados} duplicados ({porcentaje}%) sobre {total} filas")

# Aplicar la verificaci√≥n a todos los DataFrames
print("üìã Revisi√≥n general de duplicados:\n")
for nombre, df in dataframes_filtrados.items():
    revisar_duplicados(df, nombre)



### Revisi√≥n y eliminaci√≥n de filas duplicadas

Como parte del proceso de limpieza, se revisaron todos los DataFrames filtrados para detectar la presencia de **duplicados exactos**, es decir, filas que son completamente id√©nticas en todas sus columnas.

**Motivos para eliminar duplicados exactos:**
- No aportan nueva informaci√≥n.
- Pueden inflar promedios, conteos u otros c√°lculos estad√≠sticos.
- Suelen deberse a errores de carga o duplicaci√≥n accidental de registros.

**Metodolog√≠a aplicada:**
- Se revisaron todos los DataFrames (`draft`, `team_info`, `game`, `game_summary`, `other_stats`, `play_by_play`, `player`, `common_player_info`).
- Para cada uno se report√≥ el n√∫mero total de duplicados exactos.
- Se eliminaron √∫nicamente aquellos duplicados completos, sin afectar filas que contuvieran alguna variaci√≥n.

**Importante:** Este proceso no elimina filas similares con diferencias m√≠nimas (como nulos en algunas columnas). En esos casos, se recomienda una inspecci√≥n manual m√°s detallada antes de decidir si deben conservarse o no.

Este paso asegura que el an√°lisis posterior no se vea distorsionado por datos redundantes.


In [ ]:
# --- Diccionario con todos los DataFrames filtrados ---
dataframes_filtrados = {
    "draft": draft_filtrado,
    "team_info": team_info_filtrado,
    "game": game_filtrado,
    "game_summary": game_summary_filtrado,
    "other_stats": other_stats_filtrado,
    "play_by_play": play_by_play_filtrado,
    "player": player_filtrado,
    "common_player_info": common_info_filtrado,
    "MVP": MVP_filtrado,
    "Salarios": Salarios_filtrado
}

# --- Revisi√≥n y borrado de duplicados exactos ---
print("üìã Revisi√≥n de duplicados exactos y limpieza opcional:\n")

# Nuevos dataframes sin duplicados
dataframes_limpios = {}

for nombre, df in dataframes_filtrados.items():
    total = df.shape[0]
    duplicados = df.duplicated().sum()
    
    print(f"- {nombre}: {duplicados} duplicados exactos de {total} filas")
    
    # Si hay duplicados, los eliminamos (puedes comentar esta l√≠nea si quer√©s revisar primero)
    df_limpio = df.drop_duplicates().copy()
    
    # Guardar versi√≥n limpia
    dataframes_limpios[nombre] = df_limpio

# --- Reasignar a las variables originales si est√°s conforme ---
draft_filtrado = dataframes_limpios["draft"]
team_info_filtrado = dataframes_limpios["team_info"]
game_filtrado = dataframes_limpios["game"]
game_summary_filtrado = dataframes_limpios["game_summary"]
other_stats_filtrado = dataframes_limpios["other_stats"]
play_by_play_filtrado = dataframes_limpios["play_by_play"]
player_filtrado = dataframes_limpios["player"]
common_info_filtrado = dataframes_limpios["common_player_info"]
